<a href="https://colab.research.google.com/github/AgustinBatistelli/programacion_concurrente/blob/master/TP2_M2_Java_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%%writefile Hilos.java
import java.util.Random;

public class Hilos {

    public static int[][] multiplicarMatircesHilos(int[][] matrizA, int[][] matrizB, int tamanio) throws InterruptedException
    {        
        int[][] resultado = new int[tamanio][tamanio];
        int numHilos = tamanio;
        Thread[] hilos = new Thread[numHilos];
        
        for (int i = 0; i < numHilos; i++)
        {
            hilos[i] = new Thread(new MultiplicacionHilos(matrizA, matrizB, resultado,i));
            hilos[i].start();
        }

        for (Thread hilo : hilos)
            hilo.join();

        return resultado;
    }

    private static class MultiplicacionHilos implements Runnable
    {
        private int[][] matrizA;
        private int[][] matrizB;
        private int[][] resultado;
        private int fila;

        public MultiplicacionHilos(int[][] a, int[][] b, int[][] resultado, int fila)
        {
            this.matrizA = a;
            this.matrizB = b;
            this.resultado = resultado;
            this.fila = fila;
        }

        @Override
        public void run() {
            int tamanio = matrizA.length;
            for (int i = 0; i < tamanio; i++)
            {
                int suma = 0;
                for(int j = 0; j < tamanio; j++)
                {
                    suma += matrizA[fila][j] * matrizB[j][i];
                }
                resultado[fila][i] = suma;
            }
        }
    }

    public static int[][] mutliplacarMatricesSecuencial(int[][] matrizA, int[][] matrizB)
    {
        int filasMatrizA = matrizA.length;
        int columnasMatrizA = matrizA[0].length;
        int filasMatrizB = matrizB.length;
        int columnasMatrizB = matrizB[0].length;

        if (columnasMatrizA != filasMatrizB)
        {
            throw new IllegalArgumentException("No se puede realizar la multiplicacion de matrices");
        }

        int[][] resultado = new int[filasMatrizA][columnasMatrizB];
        for(int i = 0; i < filasMatrizA; i++)
        {
            for(int j = 0; j < columnasMatrizB; j++)
            {
                for(int k = 0; k < columnasMatrizA; k++)
                {
                    resultado[i][j] += matrizA[i][k] * matrizB[k][j];
                }   
            }
        }
        return resultado;
    } 

    public static void cargarMatriz(int[][] matriz, int tamanio)
    {
        Random random = new Random();
        for (int i = 0; i < tamanio; i++)
        {
            for (int j = 0; j < tamanio; j++)
                matriz[i][j] = random.nextInt(65) - 32;
        }
    }

    public static void mostrarMatriz(int[][] matriz)
    {
        for (int i = 0; i < matriz.length; i++)
        {
            for (int j = 0; j < matriz.length; j++)
                System.out.print(matriz[i][j] + " ");
            System.out.println();
        }    
    }

    public static void inicializarMatriz(int[][] matriz)
    {
        for (int i = 0; i < matriz.length; i++)
        {
            for (int j = 0; j < matriz.length; j++)
            {
                matriz[i][j] = 0;
            }
        }
    }

    public static boolean sonMatricesIguales(int[][] matrizA, int[][] matrizB)
    {
        for (int i = 0; i < matrizA.length; i++)
        {
            for (int j = 0; j < matrizA.length; j++)
            {
                if (matrizA[i][j] != matrizB[i][j])
                {
                    return false;
                }
            }
        }
        return true;
    }

    public static void mostrarResultado(int[][] matrizA, int[][] matrizB, int[][] matrizCS, int[][] matrizCH)
    {
        System.out.println("Matriz A: ");
        mostrarMatriz(matrizA);        
        System.out.println("");

        System.out.println("Matriz B: ");
        mostrarMatriz(matrizB);
        System.out.println("");

        System.out.println("Matriz CS: ");
        mostrarMatriz(matrizCS);   
        System.out.println("");

        System.out.println("Matriz CH: ");
        mostrarMatriz(matrizCH);   
        System.out.println("");
    }


    public static void main(String[] args) throws Exception {
        if (args.length != 1)
        {   
            System.out.println("No se ha especificado ningun argumento que indique el tamaño de la matriz.");
            return;
        } 
        
        int tamanio = Integer.parseInt(args[0]);

        if (tamanio <= 5 || tamanio >= 20)
        {
            System.out.println("El tamaño de la matriz debe estar entre 5 y 20.");
            return;
        }

        int[][] matrizA = new int[tamanio][tamanio];
        int[][] matrizB = new int[tamanio][tamanio];
        
        cargarMatriz(matrizA, tamanio);
        cargarMatriz(matrizB, tamanio);
        
        int[][] matrizCS = new int[tamanio][tamanio];
        int[][] matrizCH = new int[tamanio][tamanio];
        inicializarMatriz(matrizCS);
        inicializarMatriz(matrizCH);

        matrizCS = mutliplacarMatricesSecuencial(matrizA, matrizB);

        try
        {
          matrizCH = multiplicarMatircesHilos(matrizA, matrizB, tamanio);
        } catch(InterruptedException e)
        {
            System.out.println("Error al multiplicar las matrices: " + e.getMessage());
            return;
        }


        mostrarResultado(matrizA, matrizB, matrizCS, matrizCH);
        
        if (sonMatricesIguales(matrizCS, matrizCH))
        {
            System.out.println("La matrizCS es igual que la matrizCH");
        }
        else
        {
            System.out.println("La matrizCS no es igual que la matrizCH");
        }        
    }
}


Overwriting Hilos.java


In [12]:
!javac Hilos.java

In [13]:
!java Hilos 7

Matriz A: 
-29 15 6 10 23 29 16 
14 -8 19 5 -24 2 1 
-31 22 29 10 -8 -5 -2 
10 -13 -5 24 12 14 -9 
-26 11 3 -10 5 -20 -32 
-14 13 -6 -22 31 11 23 
-24 -7 -2 31 2 -12 3 

Matriz B: 
25 -31 20 10 -29 -15 7 
1 9 18 -32 0 -17 26 
-27 -17 20 8 -18 -25 31 
-27 -32 -24 -2 5 9 -7 
-17 -9 24 17 22 8 16 
11 4 24 -26 11 8 19 
4 11 -13 30 -14 -11 0 

Matriz CS: 
-1150 697 610 -625 1384 360 1222 
128 -754 -145 108 -1243 -691 98 
-1733 376 -170 -868 224 -626 961 
-362 -1261 31 -2 464 719 -133 
-883 697 34 -923 988 270 -33 
105 1375 1071 371 885 24 913 
-1544 -310 -1669 342 757 695 -825 

Matriz CH: 
-1150 697 610 -625 1384 360 1222 
128 -754 -145 108 -1243 -691 98 
-1733 376 -170 -868 224 -626 961 
-362 -1261 31 -2 464 719 -133 
-883 697 34 -923 988 270 -33 
105 1375 1071 371 885 24 913 
-1544 -310 -1669 342 757 695 -825 

La matrizCS es igual que la matrizCH
